In [12]:
import torch
from models import upvote_predictor, skipgram
import pandas as pd
import os
from util import artifacts, cache
import tokenization
import embeddings
import numpy as np

import importlib
importlib.reload(artifacts)

<module 'util.artifacts' from '/Users/cameron/Documents/mlx/hackernews/util/artifacts.py'>

In [2]:
loaded_artifacts = artifacts.load_artifacts()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [4]:
state_dict = loaded_artifacts['predictor_state']
vocab = loaded_artifacts['vocab']

model = upvote_predictor.Model(skipgram.EMBEDDING_DIM)

model.load_state_dict(state_dict)
model.eval()

Model(
  (layers): Sequential(
    (0): Linear(in_features=65, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [11]:
items_table = "hacker_news.items"

hn_test_posts = cache.query("hn_posts_for_test", f"""SELECT
    title,
    karma,
    score
    FROM {items_table}
    INNER JOIN hacker_news.users u ON {items_table}.by = u.id
    WHERE type = 'story' AND title IS NOT null
    OFFSET 10000000
    LIMIT 10
""")

hn_test_posts.dropna(inplace=True)

hn_test_posts['embeddings'] = hn_test_posts['title'].apply(embeddings.get_embeddings_for_title)

hn_test_posts.dropna()

artifacts = artifacts.load_artifacts()

embed_scaler = artifacts['embed-scaler']
karma_scaler = artifacts['karma-scaler']

title_embeddings = np.vstack(np.array(hn_test_posts['embeddings'].values))
title_embeddings = embed_scaler.transform(title_embeddings)

karma = hn_test_posts['karma'].values.reshape(-1, 1)
karma = karma_scaler.transform(karma)

def eval_test_row(row):
    embeddings = row['embeddings']
    karma = row['karma']
    return model(embeddings, karma)

hn_test_posts['predicted_score'] = hn_test_posts.apply(eval_test_row)
hn_test_posts

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)